In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import ta

In [2]:
#assign dates to dataframe index
MAR_df = pd.read_csv("/Users/fahad/project_repo/data/final/MAR/MAR_normalised.csv", index_col=0)
dates = MAR_df.index
MAR_df['Date'] =dates



In [3]:
MAR_df['Date'] = pd.to_datetime(MAR_df.Date, format ="%Y-%m-%d")

In [4]:
MAR_2010 = pd.read_csv("/Users/fahad/project_repo/data/final/MAR/MAR_10yr.csv", index_col=0)

In [9]:
#ADD LABELS TO THE TRAINING AND TEST DATAFRAMES AND RETURN DICTIONARIES WITH DATE:LABEL FORMAT
MAR_2010_labels = labelling(MAR_2010)

In [10]:
MAR_2010_flattened= get_ndays_monthly_flattened(MAR_2010,MAR_df,30)

In [11]:
MAR_2010_labels_df = pd.Series(MAR_2010_labels).to_frame("Label")

In [12]:
#CREATE DATAFRAME FROM FLATTENED DICTIONARY OF ARRAYS FOR TRAINING AND TEST DATA
MAR_2010_flat_df = pd.DataFrame.from_dict(MAR_2010_flattened).T

In [13]:
#JOIN THE LABEL DATAFRAME AND THE FLATTENED DATAFRAME ON THE INDEX WHICH IS THE DATE
MAR_2010_flat_df = MAR_2010_flat_df.merge(MAR_2010_labels_df, left_index=True, right_index=True)

In [14]:
MAR_2010_flat_df

,0,1,2,3,4,5,6,7,8,9,...,351,352,353,354,355,356,357,358,359,Label
2019-07-31,0.868454,0.873202,0.845211,0.843985,0.086897,0.000000,0.242317,0.315166,0.318182,0.227245,...,0.953948,0.018834,0.589559,0.217588,0.897615,0.910803,0.470613,0.0,0.0,1
2019-06-28,0.941887,0.943132,0.946645,0.947011,0.032748,0.844754,0.359678,0.315166,0.318182,0.549917,...,0.903838,0.060113,0.357857,0.132525,0.928059,0.944980,0.494486,0.0,0.0,1
2019-05-31,0.909406,0.923833,0.917143,0.928413,0.075742,0.364121,0.164652,0.896955,0.942932,0.479460,...,0.885756,0.073549,0.692531,0.409811,0.963255,0.955020,0.485298,0.0,0.0,0
2019-04-30,0.826840,0.826754,0.827302,0.827749,0.040651,0.215610,0.089768,0.960295,0.955020,0.494438,...,0.910996,0.040422,0.551024,0.216219,0.893544,0.945743,0.470041,0.0,0.0,1
2019-03-29,0.913309,0.911139,0.916475,0.911292,0.041977,0.560713,0.368339,0.921603,0.945743,0.550133,...,0.830037,0.102516,0.427710,0.213483,0.933555,0.940361,0.408558,0.0,0.0,0
2019-02-28,0.808942,0.817655,0.815858,0.823542,0.058459,0.158152,0.033069,0.908523,0.940361,0.460138,...,0.783247,0.072003,0.234928,0.178564,0.884643,0.925667,0.556119,0.0,0.0,1
2019-01-31,0.848494,0.849024,0.848315,0.844576,0.089956,0.697124,0.391166,0.929150,0.930205,0.473765,...,0.720959,0.072130,0.446715,0.262131,0.863749,0.916269,0.430940,0.0,0.0,0
2018-12-31,0.731371,0.739984,0.732184,0.746494,0.106157,0.240608,0.174769,0.860101,0.916269,0.458606,...,0.702362,0.098614,0.566692,0.340377,0.789727,0.832570,0.603165,0.0,0.0,0
2018-11-30,0.697798,0.701424,0.697852,0.699262,0.048419,0.473793,0.297390,0.824807,0.878353,0.510315,...,0.766125,0.053151,0.501206,0.212285,0.714976,0.801486,0.639368,0.0,0.0,1
2018-10-31,0.802239,0.806428,0.803002,0.807011,0.043673,0.744797,0.513166,0.749778,0.811606,0.755981,...,0.758672,0.090729,0.570126,0.163092,0.776950,0.772048,0.724928,0.0,0.0,1


In [15]:
export = MAR_2010_flat_df.to_csv("/Users/fahad/project_repo/data/final/MAR/MAR_2010_pre_signals.csv")

In [7]:
def labelling(dataframe):
    """This generates labels for the dataframes we wish to predict price.
    A label of 1 suggests an increase in price between months while a label of 0 indicates
    a decrease in price. If the current price is greater than the previous price then a"""
    empty_dict = dict()
    prev_close =0
    counter = 0
    for index,cols in dataframe.iterrows():
        counter +=1
        current_close = cols['Close']
        if float(prev_close) <= float(current_close):
            #signals an increase in price between last month and next month
            empty_dict[index] = 1
        elif float(prev_close) >= float(current_close):
            empty_dict[index]=0

        #reassign prev close to the current close     
        prev_close = current_close
    
    return empty_dict

In [8]:
def get_ndays_monthly_flattened(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily"""
    empty_dict = dict()
    count = 0
    for end in dataframe_period_end.index:
#         print ("month end/:%s" % end)
        
        month_key = dataframe_period_end.index[count+1]
        
#         print ("month key:%s" % month_key)
        
        last_index = dataframe_period_end.index[-1]
        
#         print ("last index:%s " % last_index)
        
        if (month_key != last_index):
            temp_df = dataframe_subperiod.loc[dataframe_subperiod['Date']<end].head(n)
#             print ("\n")
#             print (temp_df.head(5))
            temp_df = temp_df.drop(['Date'], axis=1)
            
            if (len(temp_df.values.flatten())==360):
                empty_dict[month_key]= temp_df.values.flatten()
                count +=1      
            else:
                pass

        else:
            pass
       
    return empty_dict
    